# COMP5329 - Assignment 2

In [14]:
# import google
import collections
import json
import re

import nltk
import numpy as np
import pandas
import pandas as pd

# import torchtext
import PIL.Image
import torch
import torch.nn as nn
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MOUNT_PATH = '/content/drive'
# DRIVE_PATH = f'{MOUNT_PATH}/My Drive'
# PROJECT_PATH = DRIVE_PATH + "/Assignment 2"
PROJECT_PATH = "./"
IMG_PATH = f"{PROJECT_PATH}/data"
TRAIN_CSV_PATH = f"{PROJECT_PATH}/train.csv"
TEST_CSV_PATH = f"{PROJECT_PATH}/test.csv"

# google.colab.drive.mount(MOUNT_PATH)
torch.cuda.empty_cache()

In [15]:
# from efficientnet_pytorch import EfficientNet

# model = EfficientNet.from_pretrained("efficientnet-b0", include_top=False)

## Dataset


### Train dataset

In [16]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TRAIN_CSV_PATH, names=range(4), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[3] = self.df_data[3].fillna("")
        self.df_data[2] += self.df_data[3]
        self.df_data = self.df_data.drop(3, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "label", 2: "caption"}, axis=1).dropna()

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Test dataset

In [4]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[2] = self.df_data[2].fillna("")
        self.df_data[1] += self.df_data[2]
        self.df_data = self.df_data.drop(2, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        caption = self.df_data.iloc[idx, 1]

        sample = {"caption": caption, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Preprocessing

### Vocab

### Dataset pre-transformations 

In [5]:
def one_hot_encode_labels(dataset):
    if "one_hot_encoded_labels" not in dataset.tags:
        dataset.df_data["label"] = dataset.df_data["label"].apply(
            lambda l: torch.nn.functional.one_hot(
                torch.tensor([int(i) - 1 if int(i) < 12 else int(i) - 2 for i in l.split(" ")]), 18
            )
            .sum(axis=0)
            .float()
        )

        dataset.tags.add("one_hot_encoded_labels")

## Modules


### Caption embedding

In [17]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
lang_model = AutoModel.from_pretrained("prajjwal1/bert-mini")

for name, param in lang_model.named_parameters():
    param.requires_grad_(False)

Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Pretrained model surgery

In [7]:
import timm
from efficientnet_pytorch import EfficientNet

# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
features = timm.create_model("tf_efficientnetv2_s_in21k", pretrained=True, num_classes=0)

for name, param in features.named_parameters():
    param.requires_grad_(False)
#     print(name)

conv_stem.weight
bn1.weight
bn1.bias
blocks.0.0.conv.weight
blocks.0.0.bn1.weight
blocks.0.0.bn1.bias
blocks.0.1.conv.weight
blocks.0.1.bn1.weight
blocks.0.1.bn1.bias
blocks.1.0.conv_exp.weight
blocks.1.0.bn1.weight
blocks.1.0.bn1.bias
blocks.1.0.conv_pwl.weight
blocks.1.0.bn2.weight
blocks.1.0.bn2.bias
blocks.1.1.conv_exp.weight
blocks.1.1.bn1.weight
blocks.1.1.bn1.bias
blocks.1.1.conv_pwl.weight
blocks.1.1.bn2.weight
blocks.1.1.bn2.bias
blocks.1.2.conv_exp.weight
blocks.1.2.bn1.weight
blocks.1.2.bn1.bias
blocks.1.2.conv_pwl.weight
blocks.1.2.bn2.weight
blocks.1.2.bn2.bias
blocks.1.3.conv_exp.weight
blocks.1.3.bn1.weight
blocks.1.3.bn1.bias
blocks.1.3.conv_pwl.weight
blocks.1.3.bn2.weight
blocks.1.3.bn2.bias
blocks.2.0.conv_exp.weight
blocks.2.0.bn1.weight
blocks.2.0.bn1.bias
blocks.2.0.conv_pwl.weight
blocks.2.0.bn2.weight
blocks.2.0.bn2.bias
blocks.2.1.conv_exp.weight
blocks.2.1.bn1.weight
blocks.2.1.bn1.bias
blocks.2.1.conv_pwl.weight
blocks.2.1.bn2.weight
blocks.2.1.bn2.bias
block

In [8]:
from torchinfo import summary

summary(features, (1, 3, 224, 224), device="cpu")

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  --                        --
├─Conv2dSame: 1-1                             [1, 24, 112, 112]         (648)
├─BatchNorm2d: 1-2                            [1, 24, 112, 112]         (48)
├─SiLU: 1-3                                   [1, 24, 112, 112]         --
├─Sequential: 1-4                             [1, 256, 7, 7]            --
│    └─Sequential: 2-1                        [1, 24, 112, 112]         --
│    │    └─ConvBnAct: 3-1                    [1, 24, 112, 112]         (5,232)
│    │    └─ConvBnAct: 3-2                    [1, 24, 112, 112]         (5,232)
│    └─Sequential: 2-2                        [1, 48, 56, 56]           --
│    │    └─EdgeResidual: 3-3                 [1, 48, 56, 56]           (25,632)
│    │    └─EdgeResidual: 3-4                 [1, 48, 56, 56]           (92,640)
│    │    └─EdgeResidual: 3-5                 [1, 48, 56, 56]       

### Combined model

In [9]:
NUM_ClASSES = 18
import math

import torch.nn as nn

img_dim = 1280
word_dim = 256


class Combined_Model(torch.nn.Module):
    def __init__(self, visual_features, lang_model):
        super().__init__()
        self.visual_features = visual_features
        self.lang_model = lang_model
        self.proj = nn.Linear(img_dim, word_dim)  # (feature.shape, hidden.shape)
        self.scale = math.sqrt(word_dim)
        self.activation = nn.ReLU()
        self.combine = nn.Linear(
            word_dim + img_dim + word_dim, NUM_ClASSES
        )  # (feature.shape + hidden.shape + hidden.shape)
        #         self.hidden1 = nn.Linear(256, 128)

    #         self.output = nn.Linear(256, NUM_ClASSES)

    def forward(self, images, input_ids, attention_mask):  # requires tokenized captions
        images = self.visual_features(images)
        proj = self.proj(images)

        lang_outs = self.lang_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden = lang_outs["last_hidden_state"]
        pooler_out = lang_outs["pooler_output"]

        attention_weights = (
            torch.bmm(hidden, proj.unsqueeze(-1)).squeeze(-1) / self.scale
        )  # (batch, seq_len, 1)
        attention_out = torch.bmm(attention_weights.unsqueeze(1), hidden).squeeze(1)

        out = torch.cat([images, attention_out, pooler_out], dim=-1)
        out = self.combine(out)
        #         out = self.activation(out)
        #         out = self.hidden1(out)
        #         out = self.activation(out)
        #         out = self.output(out)
        #         out = self.activation(out)
        return out

## Transforms and training support

In [10]:
class FieldTransform(object):
    def __init__(self, field, transform):
        self.field = field
        self.transform = transform

    def __call__(self, sample):
        sample[self.field] = self.transform(sample[self.field])
        return sample

## Experiments

In [11]:
def train_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["label"] = torch.stack(X["label"])
    X["image"] = torch.stack(X["image"])

    return X


normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms = torchvision.transforms.Compose(
    [
        #   FieldTransform('image', torchvision.transforms.Resize((64, 64))),
        #   FieldTransform('image', torchvision.transforms.CenterCrop(64)),
        FieldTransform("image", torchvision.transforms.Resize((224, 224))),
        FieldTransform("image", torchvision.transforms.ToTensor()),
        FieldTransform("image", normalize),
    ]
)

In [12]:
train_data = TrainDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(train_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
trainds, valds = random_split(train_data, [27000, 3000])

train_dl = DataLoader(
    trainds, batch_size=32, shuffle=True, collate_fn=train_collate_fn, num_workers=24
)

val_dl = DataLoader(valds, batch_size=8, shuffle=False, collate_fn=train_collate_fn, num_workers=24)

# model = Combined().to(DEVICE)
model = Combined_Model(features, lang_model).to(device)  # call fp16 for model after training!
# optim = torch.optim.Adam(model.parameters(), lr=0.001)

from sam import SAM

base_optimizer = torch.optim.Adam  # define an optimizer for the "sharpness-aware" update
optimizer = SAM(model.parameters(), base_optimizer, lr=0.005)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCEWithLogitsLoss()

# torch.save(model, "test_v7.5.pt")

In [13]:
import os
from copy import deepcopy

from sklearn.metrics import f1_score
from tqdm import tqdm

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


val_mean_f1 = []
val_sample_f1 = []
model_in_memory = {}

scaler = torch.cuda.amp.GradScaler()
for epoch in range(20):
    train_loss = []
    train_outs = []
    train_lables = []
    model.train()

    # fine tune after 5 epoches
    if epoch == 0:
        finetune_param = False
        for name, param in model.visual_features.named_parameters():
            #     print(name.split(".")[1])
            if "blocks.5.7" in name:  # 22; required grad for last 2 blocks
                finetune_param = True
            if finetune_param:
                param.requires_grad_(True)

        finetune_param = False
        for name, param in model.lang_model.named_parameters():
            if "pooler" in name.split("."):
                finetune_param = True
            if finetune_param:
                param.requires_grad_(True)
        for name, param in model.named_parameters():
            if param.requires_grad:
                print(name)

    for i, batch in enumerate(tqdm(train_dl)):
        #         optim.zero_grad()
        optimizer.zero_grad()
        captions = batch["caption"]
        images = batch["image"].to(device)
        labels = batch["label"].to(device)

        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        with torch.cuda.amp.autocast():
        # first second step
       
        
            predictions = model(images, input_ids, attention_mask)
            loss = criterion(predictions, labels)
#             loss.backward()
    #         optimizer.first_step(zero_grad=True)
#             optimizer.step()

        #         with torch.cuda.amp.autocast():
        #             # second step
#         predictions = model(images, input_ids, attention_mask)
#         loss = criterion(predictions, labels)
#         loss.backward()
#         optimizer.second_step(zero_grad=True)
#         #
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss.append(loss.item())

    val_loss = []
    val_outs = []
    val_labels = []

    model.eval()
    for i, batch in enumerate(val_dl):

        captions = batch["caption"]
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        predictions = model(images, input_ids, attention_mask)
        loss = criterion(predictions, labels)
        val_loss.append(loss.item())
        val_outs.append(predictions.detach().cpu().numpy())
        val_labels.append(labels.detach().cpu().numpy())
    val_labels = np.vstack(val_labels)
    val_outs = np.vstack(val_outs)
    mean_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="micro")  # mean f1
    sample_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="samples")  # mean f1

    if len(val_sample_f1) == 0 or sample_f1 > max(val_sample_f1):
        print("saving best")
        try:
            del model_in_memory["best"]
        except:
            pass
        torch.cuda.empty_cache()
        model.to("cpu")
        model_in_memory["best"] = deepcopy(model.state_dict())
        model.to(device)
    if epoch % 10 == 0:
        torch.save(model_in_memory["best"], "test_v7.5.pt")

    val_mean_f1.append(mean_f1)
    val_sample_f1.append(sample_f1)
    cur_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch:{epoch}, train/val loss:{round(np.mean(train_loss),7),round(np.mean(val_loss),7)},sample/mean f1:{round(sample_f1, 7), round(mean_f1, 7)}, lr:{cur_lr}"
    )

  0%|          | 0/844 [00:00<?, ?it/s]

visual_features.blocks.5.7.conv_pw.weight
visual_features.blocks.5.7.bn1.weight
visual_features.blocks.5.7.bn1.bias
visual_features.blocks.5.7.conv_dw.weight
visual_features.blocks.5.7.bn2.weight
visual_features.blocks.5.7.bn2.bias
visual_features.blocks.5.7.se.conv_reduce.weight
visual_features.blocks.5.7.se.conv_reduce.bias
visual_features.blocks.5.7.se.conv_expand.weight
visual_features.blocks.5.7.se.conv_expand.bias
visual_features.blocks.5.7.conv_pwl.weight
visual_features.blocks.5.7.bn3.weight
visual_features.blocks.5.7.bn3.bias
visual_features.blocks.5.8.conv_pw.weight
visual_features.blocks.5.8.bn1.weight
visual_features.blocks.5.8.bn1.bias
visual_features.blocks.5.8.conv_dw.weight
visual_features.blocks.5.8.bn2.weight
visual_features.blocks.5.8.bn2.bias
visual_features.blocks.5.8.se.conv_reduce.weight
visual_features.blocks.5.8.se.conv_reduce.bias
visual_features.blocks.5.8.se.conv_expand.weight
visual_features.blocks.5.8.se.conv_expand.bias
visual_features.blocks.5.8.conv_pwl

100%|██████████| 844/844 [00:48<00:00, 17.46it/s]


saving best


  0%|          | 0/844 [00:00<?, ?it/s]

...

Epoch:10, train/val loss:(0.017863, 0.1080543),sample/mean f1:(0.8510242, 0.8234495), lr:5e-05


  0%|          | 0/844 [00:00<?, ?it/s]

Epoch:11, train/val loss:(0.01389, 0.1176253),sample/mean f1:(0.8488902, 0.8214408), lr:5e-05


 75%|███████▍  | 631/844 [02:03<00:41,  5.09it/s]


KeyboardInterrupt: 

In [ ]:
# try gradient clipping

In [ ]:
# effv2 + tiny (99.9mb) finetune from 0; 5.12, 5e-05: 85.7
# effv2 + tiny (99.9mb) finetune from 0; 5.7, 5e-05: 85.7

In [ ]:
torch.save(model_in_memory["best"], "test_v7.5.pt")

In [ ]:
# stopper for run all
raise ValueError

# correction

In [ ]:
model = Combined_Model(features, lang_model)
model.load_state_dict(torch.load("test_v7.5.pt"))
model.to(device)

In [ ]:
# cheat / or use val dataset. Want
class CheatDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pd.read_csv("test_cheat.csv", names=range(3), skiprows=1)[[0, 2, 1]].dropna()
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data = self.df_data.rename({0: "image", 2: "label", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample


cheat_data = CheatDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(cheat_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
cheat_dl = DataLoader(
    cheat_data, batch_size=8, shuffle=False, collate_fn=train_collate_fn, num_workers=24
)

from sklearn.metrics import precision_recall_curve

# use the same validation set/training set
val_outs = []
val_labels = []
model.eval()

for i, batch in enumerate(cheat_dl):

    captions = batch["caption"]
    images = batch["image"].to(DEVICE)
    labels = batch["label"].to(DEVICE)

    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # predictions = model(images, captions)
    predictions = model(images, input_ids, attention_mask)

    val_outs.append(predictions.detach().cpu().numpy())
    val_labels.append(labels.detach().cpu().numpy())
val_labels = np.vstack(val_labels)
val_outs = np.vstack(val_outs)


best_thresholds = np.zeros(18)
for i in range(18):
    fpr, tpr, thresholds = precision, recall, thresholds = precision_recall_curve(
        val_labels[:, i], (val_outs)[:, i]
    )
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    best_thresholds[i] = thresholds[ix]

print(best_thresholds)
np.save("best_thresholds", best_thresholds)

In [ ]:
f1_score(y_true=val_labels, y_pred=1.0 * (val_outs > 0), average="samples")

In [ ]:
f1_score(y_true=val_labels, y_pred=1.0 * (val_outs - best_thresholds > 0), average="samples")

# generate test labels

In [ ]:
# stopper for run all
raise ValueError

In [ ]:
def test_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["image"] = torch.stack(X["image"])

    return X


test_data = TestDataset(transform=transforms)
test_dl = torch.utils.data.DataLoader(
    test_data, batch_size=16, shuffle=False, collate_fn=test_collate_fn, num_workers=0,
)


# save predictions
model.eval()
test_preds = []
for i, batch in enumerate(test_dl):
    captions = batch["caption"]
    images = batch["image"].to(DEVICE)

    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    predictions = model(images, input_ids, attention_mask)
    test_preds.append(predictions.detach().cpu().numpy())

test_preds = np.vstack(test_preds)


def out_logits_to_preds(logits, best_thresholds):
    labels = []
    logits = (logits - best_thresholds) > 0
    lables_available = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19])
    for i in range(logits.shape[0]):
        labels.append(list(lables_available[logits[i]]))
    return labels


# lables_available[(test_preds[:10] > 0)]
# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}

test_labels = out_logits_to_preds(test_preds, best_thresholds)

df_test = pd.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
test_labels_str = [" ".join([str(i) for i in labels]) for labels in test_labels]
df_test["Labels"] = test_labels_str
df_test.rename({0: "ImageID"}, axis=1, inplace=True)

df_test[["ImageID", "Labels"]].to_csv("test_predictions.csv", index=False)